# Migrate Single Database


# Initialization
You will have to run all 3 code cells in this section each time you reconnect to the cluster.

## Widget Setup
This notebook uses widgets to initialize the InventoryCollector as well as help scan a particular database easily.

In [ ]:
try:
    spark.sql('use catalog hive_metastore;')
except Exception as e:
    pass

hmsDatabaseList = [row.databaseName for row in spark.sql('show databases').select('databaseName').collect()]
if len('hmsDatabaseList') > 1024:
    print('Warning! More than 1024 HMS databases. Picker widget will only display first 1024')
dbutils.widgets.dropdown("HMS_Database", hmsDatabaseList[0], hmsDatabaseList[0:1023], "HMS Database");
dbutils.widgets.text("Inventory_Catalog",   "hive_metastore", "Write Catalog for Inventory")
dbutils.widgets.text("Inventory_Database",   "databricks_inventory", "Write Database for Inventory")
dbutils.widgets.text("Migration_Catalog", "DEST_CATALOG_HERE", "Migration Target Catalog")

print("Database list:\n")
print('\n'.join(hmsDatabaseList))

## Import Collector Library
Note: The usage of "from ... import" works expects a single .py file, as included from github.
If you are not using github repos, create a notebook with the DbInventoryCollector.py file's contents in it, and change this line to read:
`%run ./DB-Inventory-Collector`

In [ ]:
from DbInventoryCollector import InventoryCollector


## Create Collector
We must initialize the collector with a location to store its data. The .initialize() method will create the schemas and tables for the data to be stored inside.
Remember you will have to re-execute this cell every time the code for InventoryCollector is updated.

In [ ]:
#Initialize InventoryCollector
collector = InventoryCollector(spark, dbutils.widgets.get("Inventory_Catalog"), dbutils.widgets.get("Inventory_Database"))
collector.initialize()
# display(collector.get_database_inventory_summary())

# Single Database Migration

## DDL Generation

In [ ]:
selectedDatabase = dbutils.widgets.get("HMS_Database")
destCatalog = dbutils.widgets.get("Migration_Catalog")

if destCatalog == "DEST_CATALOG_HERE":
    raise Exception("You must set the destination catalog using the widgets above")

(ddl_objects, ddl_grants) = collector.generate_migration_ddl(selectedDatabase, destCatalog)

print("Finished Generation of both object and grant DDL")

## Inspect Generated DDL

In [ ]:
print(';\n\n'.join(ddl_objects))

In [ ]:
print(';\n\n'.join(ddl_grants))

## Execute DDL

In [ ]:
print("Executing Object Creation DDL")
collector.execute_sql_list(ddl_objects)
print("Executing Grants")
collector.execute_sql_list(ddl_grants)